# Numpy

Структура занятия:

1) Введение, массивы

2) Доступ к элементам и срезам

3) Выполнение вычислений

4) Индексация

5) Сортировка

6) Структуры

## Введение

Библиотека NumPy предоставляет реализации вычислительных алгоритмов (в виде функций и операторов), оптимизированные для работы с многомерными массивами, структуры NumPy эффективно хранят данные.

Вспомним как хранятся данные в Python...

Целое число в Python - это больше чем просто целое число. Целочисленный тип хранится в виде структуры [языка Си](https://github.com/python/cpython/blob/main/Objects/longobject.c#L140), содержит счётчик ссылок, кодирующий тип, фактическое целочисленное значение и т.д. Это значит, что существует достаточно большая избыточность хранения данных, в сравнении с компилируемыми языками, такими как Си. 

Анаогично, список в Python - это больше чем просто список. Список содержит указатель на блок указателей, каждый из которых, в свою очередь, указывает на целый объект языка Python, например, целое число. 

Преимущество массива Python - гибкость. Недостаток - низкая эффективность хранения данных и их обработки. 

Встроенный модуль `array` можно использовать для плотного (эффективного) хранения данных одного типа.

In [ ]:
import array

data = array.array('i', range(100))
data

In [ ]:
from sys import getsizeof

getsizeof(data), getsizeof(list(range(100)))

Массивы NumPy, помимо эффективного хранения данных, обеспечивают также возможность выполнения эффективных операций над этими данными.

In [ ]:
pip install numpy

In [ ]:
import numpy as np

np.array(range(100))

Важно отметить, что массивы NumPy могут содержать элементы только одного типа. Если типы элементов не совпадают, NumPy пытается привести элементы к одному типу

In [ ]:
np.array(list(range(10)) + [0.1])

In [ ]:
np.array(list(range(10)) + ['1'])

In [ ]:
np.array(list(range(10)) + [0.1, '1'])

Есть возможность явно задать тип массива

In [ ]:
np.array(range(10), dtype='float64')

Все типы

In [ ]:
np.sctypes

Как ещё можно создать массив:

In [ ]:
# массив нулей
np.zeros(10)

In [ ]:
# массив как range
np.arange(20, 40, 2)

In [ ]:
# 3х мерный массив из списков
np.array([[1,2,3], [4,5,6], [7,8,9]])

In [ ]:
# массив единиц
np.ones((3, 2))

In [ ]:
# диагональная матрица
np.eye(3)

In [ ]:
# матрица c равномерно распределёнными случайными значениями
np.random.random((3, 2))

In [ ]:
# матрица c нормально распределёнными случайными значениями с медианой 0 и ст. отклонением 2
np.random.normal(0, 2, (3, 2))

## Доступ к элементам

Доступ по индексам

In [ ]:
A = np.array([[1,2,3], [4,5,6], [7,8,9]])
A[1, 2]  # особенный синтаксис np

In [ ]:
A[1][2]

In [ ]:
A[1, -1]

In [ ]:
B = A.copy()  # обратите внимание, создаётся полная копия!
B[1][2] = 66
B

In [ ]:
A

In [ ]:
a = [[1,2,3], [4,5,6], [7,8,9]]  # обычный массив (список) питона
b = a.copy()  # при работе со списком, создаётся поверхностная копия
b[1][2] = 66
b

In [ ]:
a

In [ ]:
A[::-1]

Дополнительные возможности работы со срезами

In [ ]:
A[:2, :2]  # 2 строки и 2 столбца

In [ ]:
A[:2, ::2]  # 2 строки и каждый 2-й столбец

In [ ]:
A[::-1, ::-1]  # пееворачивание подмассивов

In [ ]:
A[:, 0] # первый столбец

In [ ]:
A[1, :] # вторая строка

Срезы возвращают не копии массивов, а их предсавления. Это значит что изменение помассива (среза) повлечёт за собой изменение исходного массива. Для создания копий, необходимо использовать метод `.copy()`

In [ ]:
a_sub = A[:2, :2]
a_sub

In [ ]:
a_sub[0, 0] = 100
A

Можно изменять форму массива при помощи метода `.reshape()`. При этом, размер исходного массива и преобразования должны соответствовать друг другу.

In [ ]:
# поместим числа от 1 до 10 в таблицу 3x3
np.arange(1, 10).reshape((3, 3))

In [ ]:
np.arange(12).reshape((2, 6))

При помощи методов `.concatenate()`, `.split()` массивы можно объединять и разбивать

In [ ]:
np.concatenate([A, A])

In [ ]:
np.concatenate([A, A], axis=1)

In [ ]:
np.split(A, [1, 1])  # кроме массива задаём точки раздела

In [ ]:
np.split(A, [1, 1], axis=1)  # кроме массива задаём точки раздела

## Выполнение вычислений

Ключ к ускорению обработки массивов - использование векторизованных операций.

Многие (но не все) функции NumPy работают с разблокированным GIL!

Сравим скорость обработки массива стандартным способом, то есть при помощи цикла и использование веркоризованных (направленных на вектор) операций.

In [ ]:
def compute_negative(arr):
    result = np.empty(len(arr))
    for i in range(len(arr)):
        result[i] = -arr[i]
    return result

V = np.random.random(size=100_000)
%timeit compute_negative(V)  # через цикл

In [ ]:
%timeit (-V)  # через векторизованные операции

Векторизованные операции реализованы посредством *универсальных функций*, которые позволяют быстро выполнять повторяющиеся операции на массивах NumPy 

Универсальные функции делятся на унарные и бинарные, список:
- унарные: `-`, `np.abs()`, `np.sin()`, `np.log()`, `np.exp()`, и т.д.
- бинарные: `+`, `-`, `*`, `**`, `/`, `//`, `%`, и т.д.

Арифметические операторы являются адаптерами для функций, например `+` = `np.add()`

Все универсальные функции имеют дополнительный метод `.reduce()`

In [ ]:
A = np.random.random(size=100_000)
np.add.reduce(A)

Агрегирования:

In [ ]:
# сумма
np.sum(A)

In [ ]:
# минимум и максимум
np.min(A), np.max(A)

In [ ]:
M = np.random.random((3, 3))
M

In [ ]:
np.sum(M)

In [ ]:
np.sum(M, axis=0)

In [ ]:
np.sum(M, axis=1)

Возможные агрегирования:
- `np.sum` - сумма
- `np.mean` - среднее
- `np.std` - стандартное отклонение
- `np.var` - дисперсия
- `np.median` - медиана
- `np.percentile` - квантили элементов
- `np.all` - все ли элементы истины
- `np.any` - есть ли среди элементов истинный

Транслирования - правила применения бинарных универсальных функций к массивам различной длины.

In [ ]:
M + 5  # простой случай - сложение применяется ко всем элементам

In [ ]:
M + M  # простой случай - сложение происходит попарно между элементами с одинаковыми индексами

In [ ]:
M + np.array([1, 2, 3])  # сложение происходит по столбцам

In [ ]:
M + np.array([1, 2, 3]).reshape((3, 1))  # сложение по строкам

Маски. Универсальные функции `>`, `<`, `=`, `!=` позволяют создать маски, то есть классифицировать данные по какому либо признаку, преобразовать массивы к виду истина / ложь. Это необходимо, если мы хотим ответить на какой-либо конкретный вопрос

In [ ]:
M

In [ ]:
M.mean()

In [ ]:
M > M.mean()

In [ ]:
M.sum()

In [ ]:
np.sum(M > M.mean())  # сочетание агрегации и маски

Условия можно сочетать при помощи логических операторов `&` (and), `|` (or), `^` (xor), `~` (not)

In [ ]:
(M > M.mean()) | (M == M.min())

### Пример. Статистики по росту людей

In [ ]:
import csv

heights = []

with open("weight_height.csv") as csvfile:
    r = csv.DictReader(csvfile)
    for i in r:
        heights.append(float(i['Height']))

In [ ]:
heights_arr = np.array(heights)

In [ ]:
heights_arr

In [ ]:
heights_arr.mean(), heights_arr.max(), heights_arr.min()

In [ ]:
heights_arr.std(), np.median(heights_arr), np.percentile(heights_arr, 75), np.percentile(heights_arr, 25)

Так как куда интереснее видеть графическое представление данных, воспользуемся библиотекой `matplotlib`. Она отлично интегрированна в Jupyter

In [ ]:
pip install matplotlib

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(heights_arr)

Точные численные данные

In [ ]:
np.count_nonzero(heights_arr)

In [ ]:
h_h = np.percentile(heights_arr, 98)
np.count_nonzero(heights_arr >= h_h), np.count_nonzero(heights_arr < h_h), h_h

In [ ]:
l_h = np.percentile(heights_arr, 2)
np.count_nonzero(heights_arr >= l_h), np.count_nonzero(heights_arr < l_h), l_h

## Индексация

In [ ]:
A = np.random.randint(0, 20, size=12)
A

Нужные элементы можно запрашивать напрямую, с указанием индексов 

In [ ]:
[A[1], A[5], A[3]]

Но лучшим способом является передача массива индексов

In [ ]:
idx = [1, 5, 3]
A[idx]

Такая индексация работает с многомерными массивами, при этом необходимо передавать индексы нужных измерений для каждого элемента

In [ ]:
A = np.random.randint(0, 20, size=12).reshape((4, 3))
A

In [ ]:
row = [0, 1, 3]
col = [0, 2, 2]
A[row, col]

Можно совмещать простые индексы и массивы индексов

In [ ]:
row = 1
col = [0, 2, 2]
A[row, col]

Частая сфера применения разного рода индексаций - выборка подмножеств данных, например, для разделения датасетов на обучающую и тестовую выборки.

In [ ]:
import csv

weights = []

with open("weight_height.csv") as csvfile:
    r = csv.DictReader(csvfile)
    for i in r:
        weights.append(float(i['Weight']))
        
weights_arr = np.array(weights)

In [ ]:
np.count_nonzero(heights_arr), np.count_nonzero(weights_arr)

In [ ]:
heights_arr.shape, weights_arr.shape

In [ ]:
X = np.stack([heights_arr, weights_arr], axis=-1)  # объединили рост и вес по индексу записи

In [ ]:
X.shape

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=1)

1000 случайных элементов отберём для теста, используя индексацию

In [ ]:
indexes = np.random.choice(X.shape[0], 1000, replace=False)

In [ ]:
indexes

In [ ]:
selected_test = X[indexes]
selected_test.shape

Визуализируем тестовую выборку на данных

In [ ]:
plt.scatter(X[:, 0], X[:, 1], alpha=0.1, s=1)
plt.scatter(selected_test[:, 0], selected_test[:, 1], s=1)


## Сортировка

По умолчанию используется метод timsort, хотя по желанию можно использовать heapsort, quicksort, mergesort.

In [ ]:
A = np.random.randint(0, 20, size=12)
A

In [ ]:
B = A.copy()
B.sort()
B

Также есть функция `argsort`, которая возвращает индексы отсортированных элементов

In [ ]:
C = A.copy()
x = C.argsort()
x

In [ ]:
X

С сортировкой 2-х мерных (многомерных) массивов сложнее: все строки и столбцы рассматриваются как отдельные элементы и взаимосвязи между ними теряются

In [ ]:
x_sorted = X.copy()
x_sorted.sort()  # сортируем столбцы - данные смешиваются
x_sorted

In [ ]:
x_sorted = X.copy()
x_sorted.sort(axis=0)  # сортируем строки - и так данные смешиваются
x_sorted

In [ ]:
X.T[0] > 198

In [ ]:
X[X.T[0] > 198]  # рост больше

In [ ]:
X[X.T[0] < 140]  # рост меньше

In [ ]:
X[X.T[1] > 115]  # вес больше

In [ ]:
X[X.T[1] < 35]  # вес меньше

## Структуры данных

Можно задать контейнеры с именованными полями, таким образом, связать данные в структуры

In [ ]:
types = {
    'names': ('heights', 'weights'),
    'formats': ('float32', 'float32'),
}
data = np.zeros(10000, dtype=types)
data['heights'] = heights_arr
data['weights'] = weights_arr

In [ ]:
data['heights']

In [ ]:
data[0]

In [ ]:
data.sort()

In [ ]:
data[:10]

In [ ]:
data[-10:]

Данные связяны, сортировка корректна!